### 알림 신호를 위한 랙의 좌표값 설정하기
- GUI에서 좌표값을 받아야함!

In [ ]:
# 일단 고정되어있다고 가정하자
# cam 5
# u_x1, u_y1, u_x2, u_y2 = 137, 32, 437, 189
# d_x1, d_y1, d_x2, d_y2 = 110, 253, 363, 515

# cam 6
u_x1, u_y1, u_x2, u_y2 = 866, 294, 1111, 33
l_x1, l_y1, l_x2, l_y2 = 897, 293, 1232, 547

### 모델 불러와서 예측해보기 - 사진

In [ ]:
from ultralytics import YOLO

# 저장된 모델 불러오기
num = 7
model = YOLO(f'./runs/detect/train{num}/weights/best.onnx')

In [ ]:
# 예측하기 - 사진

file = "C:/Users/kdp/Desktop/total_data_class_5/data_class_5/231231_1600_Cam6_C2 (1239).jpg"


results = model.predict(source = file, save=True, conf=0.25)

# source : 예측할 이미지, 비디오 또는 웹캠의 경로나 인덱스를 지정
# save : 예측 결과 저장 여부
# conf : 예측 결과의 신뢰도 임계값을 지정함 

In [ ]:
# 예측된 결과에서 좌표값 추출
import cv2 

for result in results:
    boxes = result.boxes.xyxy.cpu().numpy()
    class_ids = result.boxes.cls.cpu().numpy().astype(int)

    # 원본 이미지 불러오기
    img_bgr = cv2.imread(file)

    # BGR 배열을 RGB 배열로 변환 
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    for box, class_id in zip(boxes, class_ids):
        x1, y1, x2, y2 = box
        label = model.names[class_id]
        print(f"좌표: ({x1}, {y1}) - ({x2}, {y2}) 라벨: {label}")

    if label == 'Person':
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # 좌표값을 정수로 변환
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img_rgb, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # upper rack
        if (u_x1 <= x2 <= u_x2) and (u_y1 <= y2 <= u_y2):
            cv2.putText(img_rgb, 'Person on UPPER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            print('person on upper rack')

        # lower rack  
        if (l_x1 <= x2 <= l_x2) and (l_y1 <= y2 <= l_y2):
            cv2.putText(img_rgb, 'Person on LOWER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            print('person on lower rack')
        
        cv2.imwrite(f'{file}_output.jpg', img_rgb)

<hr>

In [ ]:
# 예측하기 ------------------------------------------------------------------------------

# img_file = "01frame_12696.jpg"
# results = model.predict(source = img_file, save=True, conf=0.25)


# 비디오 예측 ---------------------------------------------------------------------------

# video_file = "20231129115714.avi"
# results = model.predict(source= video_file, save=True, conf=0.25)


# 웹캠 예측  ----------------------------------------------------------------------------

# results = model.predict(source='0', save=True, conf=0.25)  # 0은 기본 웹캠을 의미




# source : 예측할 이미지, 비디오 또는 웹캠의 경로나 인덱스를 지정
# save : 예측 결과 저장 여부
# conf : 예측 결과의 신뢰도 임계값을 지정함 